# [SETUP] 

In [ ]:
! python ./setup.py

## Connect to DuckDB

In [ ]:
import duckdb
import pandas as pd

%load_ext sql
conn = duckdb.connect("tpch.db")
%sql conn --alias duckdb

In [ ]:
%%sql
show tables;

# [Data Model]

The TPC-H data represents a car parts seller’s data warehouse, where we record orders, items that make up that order (lineitem), supplier, customer, part (parts sold), region, nation, and partsupp (parts supplier). 

Note: Have a copy of the data model as you follow along; this will help in understanding the examples provided and in answering exercise questions.

![](../../tpch_erd.png)


# Using nested data types effectively

##  Use STRUCT for one-to-one & hierarchical relationships

In [ ]:
%%sql
-- without nested data type
SELECT l.*,
c.*,
s.*
FROM 
    lineitem l
LEFT JOIN 
    orders o ON l.l_orderkey = o.o_orderkey
LEFT JOIN 
    customer c ON o.o_custkey = c.c_custkey
LEFT JOIN 
    supplier s ON l.l_suppkey = s.s_suppkey
LIMIT 5;

![Cols to struct](./images/col-struct.png)

In [ ]:
%%sql
-- with nested data types
SELECT 
    l.*, 
    struct_pack(
        id := c.c_custkey,
        name := c.c_name,
        address := c.c_address,
        nationkey := c.c_nationkey,
        phone := c.c_phone,
        acctbal := c.c_acctbal,
        mktsegment := c.c_mktsegment,
        comment := c.c_comment
    ) AS customer,
    struct_pack(
        id := s.s_suppkey,
        name := s.s_name,
        address := s.s_address,
        nationkey := s.s_nationkey,
        phone := s.s_phone,
        acctbal := s.s_acctbal,
        comment := s.s_comment
    ) AS supplier
FROM 
    lineitem l
LEFT JOIN 
    orders o ON l.l_orderkey = o.o_orderkey
LEFT JOIN 
    customer c ON o.o_custkey = c.c_custkey
LEFT JOIN 
    supplier s ON l.l_suppkey = s.s_suppkey
LIMIT 5;

In [ ]:
%%sql
-- Hierarchical data 
SELECT 
    l.*, 
    struct_pack(
        id := c.c_custkey,
        name := c.c_name,
        address := c.c_address,
        nationkey := c.c_nationkey,
        phone := c.c_phone,
        acctbal := c.c_acctbal,
        mktsegment := c.c_mktsegment,
        comment := c.c_comment,
        nation := struct_pack(
            nationkey := n.n_nationkey,
            name := n.n_name,
            regionkey := n.n_regionkey,
            comment := n.n_comment
        )
    ) AS customer,
    struct_pack(
        id := s.s_suppkey,
        name := s.s_name,
        address := s.s_address,
        nationkey := s.s_nationkey,
        phone := s.s_phone,
        acctbal := s.s_acctbal,
        comment := s.s_comment,
        nation := struct_pack(
            nationkey := sn.n_nationkey,
            name := sn.n_name,
            regionkey := sn.n_regionkey,
            comment := sn.n_comment
            )
    ) AS supplier
FROM 
    lineitem l
LEFT JOIN 
    orders o ON l.l_orderkey = o.o_orderkey
LEFT JOIN 
    customer c ON o.o_custkey = c.c_custkey
LEFT JOIN
    nation n ON c.c_nationkey = n.n_nationkey
LEFT JOIN 
    supplier s ON l.l_suppkey = s.s_suppkey
LEFT JOIN
    nation sn ON s.s_nationkey = sn.n_nationkey
LIMIT 5;

In [ ]:
# exercise, to the above query add region attributes

## Use ARRAY[STRUCT] for one-to-many relationships

![Rows to List](./images/row-list.png)

In [ ]:
%%sql
WITH line_items as (
SELECT 
    l_orderkey as orderkey,
    array_agg(struct_pack(
        lineitemkey := l.l_linenumber,
        partkey := l.l_partkey,
        suppkey := l.l_suppkey,
        quantity := l.l_quantity,
        extendedprice := l.l_extendedprice,
        discount := l.l_discount,
        tax := l.l_tax,
        returnflag := l.l_returnflag,
        linestatus := l.l_linestatus,
        shipdate := l.l_shipdate,
        commitdate := l.l_commitdate,
        receiptdate := l.l_receiptdate,
        shipinstruct := l.l_shipinstruct,
        shipmode := l.l_shipmode,
        comment := l.l_comment
    )) AS lineitems
FROM 
    lineitem l 
GROUP BY 
    l_orderkey)
SELECT o.*,
len(l.lineitems) as num_lineitems,
l.lineitems
FROM orders o
LEFT JOIN line_items l
on o.o_orderkey = l.orderkey
LIMIT 5;

## Using nested data types in data processing

In [ ]:
%%sql
DROP TABLE IF EXISTS wide_orders;

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS wide_orders AS 
WITH line_items as (
SELECT 
    l_orderkey as orderkey,
    array_agg(struct_pack(
        lineitemkey := l.l_linenumber,
        partkey := l.l_partkey,
        suppkey := l.l_suppkey,
        quantity := l.l_quantity,
        extendedprice := l.l_extendedprice,
        discount := l.l_discount,
        tax := l.l_tax,
        returnflag := l.l_returnflag,
        linestatus := l.l_linestatus,
        shipdate := l.l_shipdate,
        commitdate := l.l_commitdate,
        receiptdate := l.l_receiptdate,
        shipinstruct := l.l_shipinstruct,
        shipmode := l.l_shipmode,
        comment := l.l_comment
    )) AS lineitems
FROM 
    lineitem l 
GROUP BY 
    l_orderkey)
SELECT 
    o.*,
    l.lineitems,
    struct_pack(
        id := c.c_custkey,
        name := c.c_name,
        address := c.c_address,
        nationkey := c.c_nationkey,
        phone := c.c_phone,
        acctbal := c.c_acctbal,
        mktsegment := c.c_mktsegment,
        comment := c.c_comment,
        nation := struct_pack(
            nationkey := n.n_nationkey,
            name := n.n_name,
            regionkey := n.n_regionkey,
            comment := n.n_comment
        )
    ) AS customer
FROM 
    orders o
LEFT JOIN 
    line_items l ON o.o_orderkey = l.orderkey
LEFT JOIN 
    customer c ON o.o_custkey = c.c_custkey
LEFT JOIN
    nation n ON c.c_nationkey = n.n_nationkey;

### STRUCT enables more straightforward data schema and data access

In [ ]:
%%sql
SELECT o_orderkey,
customer.name,
customer.address,
lineitems[1] as first_lineitem
FROM wide_orders
limit 2;

### Nested data types can be sorted



In [ ]:
%%sql
SELECT customer
FROM wide_orders 
GROUP BY 1
ORDER BY 1
LIMIT 5


In [ ]:
# exercise: Sort ARRAY of lineitems, how is it sorted? What do you think is the sort order based on?

### UNNEST ARRAY to rows and GROUP rows to ARRAY

In [ ]:
%%sql
-- rows to ARRAY
WITH lineitems as (SELECT 
    o.o_orderkey,
    UNNEST(o.lineitems) as line_item
FROM 
    wide_orders o),
unnested_line_items AS (
SELECT o_orderkey,
    line_item.lineitemkey,
    line_item.partkey,
    line_item.quantity
    FROM lineitems
)
SELECT o_orderkey,
array_agg(struct_pack(
        line_item_key := lineitemkey,
        part_key := partkey,
        quantity := quantity)) as lineitems
FROM unnested_line_items
GROUP BY 1
LIMIT 5;

In [ ]:
%%sql
-- ARRAY to rows
WITH
  lineitems AS (
    SELECT
      o.o_orderkey,
      UNNEST (o.lineitems) AS line_item
    FROM
      wide_orders o
  )
SELECT
  o_orderkey,
  line_item.lineitemkey,
  line_item.partkey,
  line_item.quantity
FROM
  lineitems
LIMIT
  5


### Improve OBT usability with nested data types

![Nested OBT to pre aggregate](./images/nested-preagg.png)

In [ ]:
%%sql
-- get lineitem metrics
WITH
  lineitems AS (
    SELECT
      o.o_orderkey,
      UNNEST (o.lineitems) AS line_item
    FROM
      wide_orders o
  )
SELECT
  o_orderkey,
  COUNT(line_item.lineitemkey) AS num_line_items,
  SUM(line_item.quantity) AS total_line_item_quantity
FROM
  lineitems
GROUP BY
  1
ORDER BY
  1
LIMIT
  10;


In [ ]:
%%sql
-- Get order metrics
SELECT
  o_orderdate,
  SUM(o_totalprice) AS order_total_price
FROM
  wide_orders
GROUP BY
  1
ORDER BY
  1
LIMIT
  5;

In [ ]:
# exercise: create a wide_orders_v2 table that combines all the tables in the TPCH data model

## Ensure your performance meets your expectations

In [ ]:
result = %sql EXPLAIN WITH lineitems AS ( SELECT o.o_orderkey, UNNEST (o.lineitems) AS line_item FROM wide_orders o ) SELECT o_orderkey, COUNT(line_item.lineitemkey) AS num_line_items, SUM(line_item.quantity) AS total_line_item_quantity FROM lineitems GROUP BY 1 ORDER BY 1
print(result)